In [ ]:
import stko
import stk
from collections import defaultdict

bb1 = stk.BuildingBlock(
    smiles='C1=CC(=CC=C1C2=C(N=CN2)C3=CC=C(C=C3)N)N',
    functional_groups=[stk.PrimaryAminoFactory()],
)  
bb2 = stk.BuildingBlock(
    smiles='O=C([H])C1=CC(C#CC2=CC=C(C([H])=O)C=C2)=CC(C([H])=O)=C1',
    functional_groups=[stk.AldehydeFactory()],
)
cage1= stk.ConstructedMolecule(
        topology_graph=stk.cage.FourPlusSix(
            building_blocks={
                bb1: range(4,10),
                bb2: [0, 1,2,3],
                
            },
            vertex_alignments={0: 0, 1: 0, 2: 0, 3: 0},
            optimizer=stk.MCHammer(),     
        ),
    )
cage2= cage1.with_displacement([-15,-15,0])

In [ ]:

import operator

def compute_direction(common_point, first_endpoint, second_endpoint):
    first_vector = tuple(map(operator.sub, first_endpoint, common_point))
    second_vector = tuple(map(operator.sub, second_endpoint, common_point))
    return first_vector[0]*second_vector[1] - first_vector[1]*second_vector[0]
    
def does_line_segments_intersect(first_segment, second_segment):
    d1 = compute_direction(first_segment[0], first_segment[1], second_segment[0])
    d2 = compute_direction(first_segment[0], first_segment[1], second_segment[1])
    d3 = compute_direction(second_segment[0], second_segment[1], first_segment[0])
    d4 = compute_direction(second_segment[0], second_segment[1], first_segment[1])
    
    if d1*d2 < 0 and d3*d4 < 0:
        return True
    pass

def does_triangles_intersect(first_triangle, second_triangle):    
    for first_triangle_side in range(3):
        first_side = [first_triangle[first_triangle_side], first_triangle[(first_triangle_side+1)%3]]
        for second_triangle_side in range(3):
            second_side =  [second_triangle[second_triangle_side], second_triangle[(second_triangle_side+1)%3]]
            if does_line_segments_intersect(first_side, second_side):
                return True
    return False

def coordinate_maker(cage1):
    building_block_atoms = {
        bb: defaultdict(list) for bb in cage1.get_building_blocks()
    }


    for atom_info in cage1.get_atom_infos():
        bb_atoms = building_block_atoms[atom_info.get_building_block()]
        bb_atoms[atom_info.get_building_block_id()].append(atom_info.get_atom().get_id())


    building_block_atoms = {
        bb: list(d.values()) for bb, d in building_block_atoms.items()
    }

    positions = cage1.get_position_matrix()
    bb1, bb2 = cage1.get_building_blocks()

    i=0
    final_list=[]
    while i < 4:
        final_list.append(tuple(positions[building_block_atoms[bb1][i][5]]))
        i+=1
    return final_list

osman= coordinate_maker(cage1)
osman2= coordinate_maker(cage2)

def sw_list(cor1,cor2,cor3):
    sweet_list=[]
    sweet_list.append(tuple(cor1))
    sweet_list.append(tuple(cor2))
    sweet_list.append(tuple(cor3))
    sweet_list=tuple(sweet_list)
    return(sweet_list)


def triangle_maker(cor_L):
    main_list=[]
    main_list.append(sw_list(cor1=cor_L[0],cor2=cor_L[2],cor3=cor_L[3]))
    main_list.append(sw_list(cor1=cor_L[0],cor2=cor_L[1],cor3=cor_L[3]))
    main_list.append(sw_list(cor1=cor_L[1],cor2=cor_L[2],cor3=cor_L[3]))
    main_list.append(sw_list(cor1=cor_L[0],cor2=cor_L[1],cor3=cor_L[2]))
    return (main_list)

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

triangle1 =  triangle_maker(osman)

triangle2 =  triangle_maker(osman2)


fig = plt.figure()
ax = fig.add_subplot(projection='3d')

ax.add_collection(Poly3DCollection(triangle1))
ax.add_collection(Poly3DCollection(triangle2))

ax.set_xlim([-20,20])
ax.set_ylim([-20,20])
ax.set_zlim([-20,20])

plt.savefig("gokay2.png")

In [ ]:

count1=1
count2=1
for i in triangle1:
    count2=1

    for j in triangle2:

        if does_triangles_intersect(i, j) == True:
            print(f"window{count1} of cage1 and window{count2} of cage2 intersects")
        #else:
        #    print(f"window{count2} of cage1 and window{count2} of cage2 does not intersect")
        count2+=1
    count1+=1